In [ ]:
import netsquid as ns
# Set qstate formalism to use density matrices.
ns.set_qstate_formalism(ns.QFormalism.DM)

# Extra exercise 1
## Secret Bit Communication Protocol
**Objective:** Enable Bob to securely send a secret bit (0 or 1) to Alice using quantum entanglement.
**Protocol Steps:**

a) **Entanglement Distribution**
  - Alice creates two entangled qubits
  - Alice keeps one qubit and sends the other to Bob
  - (This step is already present in the protocol and requires no change to be made)

b) **Bob's Secret Encoding**
  - Bob receives Alice's qubit
  - If his secret bit is **1**: Bob applies an X gate to the received qubit
  - If his secret bit is **0**: Bob does nothing to the qubit
  - Bob sends the qubit back to Alice

c) **Alice's Decoding**
  - Alice receives Bob's qubit
  - Alice measures both qubits (her original qubit and the returned qubit)
  - **Decoding rule:**
    - If measurement results are **equal** → secret bit = **0**
    - If measurement results are **different** → secret bit = **1**

### Timeline diagram
![title](img/exercise1_timeline.drawio.png)

# Exercise starting point:

## Network setup

In [ ]:
from netsquid.nodes import Node
from netsquid.components import Port, QuantumChannel
from netsquid.nodes import DirectConnection
from netsquid.components.models import FixedDelayModel, DepolarNoiseModel

alice_node = Node("Alice")
bob_node   = Node("Bob")

alice_port = Port("Alice_port", alice_node)
bob_port   = Port("Bob_port",   bob_node)

delay_model = FixedDelayModel(delay=10.)
error_model = DepolarNoiseModel(depolar_rate=0.2, time_independent=True)

channel_a_to_b = QuantumChannel("Channel Alice to Bob",
                                  models={"delay_model": delay_model,
                                          "quantum_noise_model": error_model}
                                  )

channel_b_to_a = QuantumChannel("Channel Bob to Alice",
                                  models={"delay_model": delay_model,
                                          "quantum_noise_model": error_model}
                                  )


connection = DirectConnection(name="connection",
                              channel_AtoB=channel_a_to_b,
                              channel_BtoA=channel_b_to_a)


alice_node.connect_to(remote_node=bob_node, connection=connection,
                     local_port_name="Alice_port", remote_port_name="Bob_port")

## Protocols

In [ ]:
from netsquid.protocols import NodeProtocol

class AliceProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Alice_port"]

        q1, q2 = ns.qubits.create_qubits(2)
        
        ns.qubits.assign_qstate([q1, q2], ns.b00)
        
        port.tx_output(q2)
        print(f"{ns.sim_time()} ns Alice send q2")

        # Wait 30 ns
        yield self.await_timer(30)

        m, _ = ns.qubits.measure(q1)

        print(f"{ns.sim_time()} ns Alice measures q1: {m}")



class BobProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Bob_port"]

        yield self.await_port_input(port)
        
        q2 = port.rx_input().items[0]

        dm = ns.qubits.reduced_dm(q2.qstate.qubits)
        print(f"{ns.sim_time()} ns Bob receives q2, DM=\n{dm}")

        m, _ = ns.qubits.measure(q2)

        print(f"{ns.sim_time()} ns Bob measures q2: {m}")

alice_protocol = AliceProtocol(node=alice_node)
bob_protocol = BobProtocol(node=bob_node)

## Run simulation

In [ ]:
# Start all protocols
alice_protocol.start()
bob_protocol.start()

sim_stats = ns.sim_run()

print(sim_stats)

# Reset simulation
alice_protocol.stop()
bob_protocol.stop()

ns.sim_reset()